In [1]:
#i "nuget:https://pkgs.dev.azure.com/dnceng/public/_packaging/dotnet5/nuget/v3/index.json" 
#i "nuget:https://pkgs.dev.azure.com/dnceng/public/_packaging/dotnet-tools/nuget/v3/index.json"
#i "nuget:https://mlnetcli.blob.core.windows.net/mlnetcli/index.json"
// #i nuget:C:\dev\dotnet-interactive-extension-lab\src\MLNetAutoML.InteractiveExtension\bin\Debug


Restore sources https://mlnetcli.blob.core.windows.net/mlnetcli/index.json https://pkgs.dev.azure.com/dnceng/public/_packaging/dotnet-tools/nuget/v3/index.json https://pkgs.dev.azure.com/dnceng/public/_packaging/dotnet5/nuget/v3/index.json

In [1]:
#r "nuget:MLNetAutoML.InteractiveExtension,0.1.1"
#r "nuget:XPlot.Plotly.Interactive,4.0.6"
#r "nuget:Microsoft.ML.AutoML,0.20.5-dev.22179.1"
#r "nuget:Microsoft.Data.Analysis,0.20.5-dev.22179.1"

Restore sources https://mlnetcli.blob.core.windows.net/mlnetcli/index.json https://pkgs.dev.azure.com/dnceng/public/_packaging/dotnet-tools/nuget/v3/index.json https://pkgs.dev.azure.com/dnceng/public/_packaging/dotnet5/nuget/v3/index.json Installed Packages Microsoft.Data.Analysis, 0.20.5-dev.22179.1 Microsoft.ML.AutoML, 0.20.5-dev.22179.1 MLNetAutoML.InteractiveExtension, 0.1.1 XPlot.Plotly.Interactive, 4.0.6

Loading extensions from `MLNetAutoML.InteractiveExtension.dll`

Loading extensions from `Microsoft.Data.Analysis.Interactive.dll`

Loading extensions from `XPlot.Plotly.Interactive.dll`

Configuring PowerShell Kernel for XPlot.Plotly integration.

Installed support for XPlot.Plotly.

In [1]:

// Import common usings.
using static Microsoft.DotNet.Interactive.Formatting.PocketViewTags;
using Microsoft.DotNet.Interactive.Formatting;
using Microsoft.Data.Analysis;
using System;
using System.Collections.Generic;
using System.IO;
using System.Linq;
using Microsoft.ML;
using Microsoft.ML.Data;
using Microsoft.ML.Trainers;
using Microsoft.ML.SearchSpace;
using Microsoft.ML.SearchSpace.Tuner;
using Microsoft.ML.Transforms.Text;
using Microsoft.ML.AutoML;
using System;
using System.Collections.Generic;
using System.Text;
using System.Text.Json;
using Microsoft.ML.Runtime;
using MLNetAutoML.InteractiveExtension;

In [1]:
var TrainDataPath = Path.Combine(Directory.GetCurrentDirectory(), "titanic.csv");

var context = new MLContext();
var df = DataFrame.LoadCsv(TrainDataPath);

var pipeline = context.Transforms.Categorical.OneHotEncoding(new[] { new InputOutputColumnPair(@"Sex", @"Sex"), new InputOutputColumnPair(@"Embarked", @"Embarked") })
                           .Append(context.Transforms.ReplaceMissingValues(new[] { new InputOutputColumnPair(@"Pclass", @"Pclass"), new InputOutputColumnPair(@"Age", @"Age"), new InputOutputColumnPair(@"SibSp", @"SibSp"), new InputOutputColumnPair(@"Parch", @"Parch"), new InputOutputColumnPair(@"Fare", @"Fare") }))
                           .Append(context.Transforms.Concatenate(@"TextFeature", @"Name", "Ticket", "Cabin"))
                           .Append(context.Transforms.Text.FeaturizeText("TextFeature", "TextFeature"))
                           .Append(context.Transforms.Concatenate(@"Features", new[] { @"Sex", @"Embarked", @"Pclass", @"Age", @"SibSp", @"Parch", @"Fare", "TextFeature" }))
                           .Append(context.Transforms.Conversion.ConvertType("Survived", "Survived", DataKind.Boolean))
						   .Append(context.Auto().BinaryClassification(labelColumnName: "Survived", useFastForest: false));

var trainTestSplit = context.Data.TrainTestSplit(df, 0.1);
var monitor = new NotebookMonitor();

            var experiment = context.Auto().CreateExperiment()
                                .SetPipeline(pipeline)
                                .SetTrainingTimeInSeconds(10)
                                .SetDataset(trainTestSplit.TrainSet, trainTestSplit.TestSet)
                                .SetEvaluateMetric(BinaryClassificationMetric.Accuracy, "Survived", "PredictedLabel")
                                .SetMonitor(monitor);
			
			monitor.SetUpdate(monitor.Display());
            var res = experiment.Run().Result;

index,Trial,Metric,Pipeline
0,0,0.6666667,Unknown=>SdcaLogisticRegressionBinary
1,1,0.7051282,Unknown=>SdcaLogisticRegressionBinary
2,2,0.82051283,Unknown=>LbfgsLogisticRegressionBinary
